In [16]:
import warnings
warnings.simplefilter("ignore", FutureWarning)

from pi import gui


gui.AnalysisGui() #debug = True )

In [26]:
df = s.get_ts(pid = pid_ls[0])
d = IcosFrame.trim_icos_meta(df.icos_meta)
d['product']

C:\Miniconda3\envs\dashboard\lib\site-packages\icoscp\cpauth\exceptions.py:53: UserWarning:


Your authentication at the ICOS Carbon Portal was unsuccessful due to: Incorrect user name or password.
Falling back to anonymous data access. Please, revisit your authentication configuration
(https://icos-carbon-portal.github.io/pylib/modules/#authentication).
Authentication will become mandatory (icoscp >= 0.1.19) for external users.



'ETC NRT Fluxes'

In [23]:
from pi import icos2latex

t = icos2latex.Translator()
t.var_unit_to_latex (df.icos_var_unit_ls)


TypeError: var_unit_to_latex() missing 1 required positional argument: 'unit'

In [ ]:
gui.AnalysisGui()

In [38]:
from pi import icos_data
from pi.icos_timeseries import IcosFrame
from icos.cpb.dobj import Dobj
s = icos_data.StationData()
df = s.stations_df

pid_ls = list(df.dobj)


ModuleNotFoundError: No module named 'icos'

In [11]:
v1 = 'CO2'
v2 = 'H'
v3 = 'H2O'
var_tuple_ls = [(v1, pid_ls[0]),(v2, pid_ls[3]), (v3, pid_ls[6]),(v2, pid_ls[0]),(v3, pid_ls[3]), (v1, pid_ls[6])]

In [37]:
do = Dobj(pid_ls[0])
df = do.data[[v1,v2]]
df

NameError: name 'Dobj' is not defined

In [14]:
group_ts(var_tuple_ls=var_tuple_ls)

{'https://meta.icos-cp.eu/objects/QLk2MGR5Zv_KPmN07Z0ZNSHe': {'cols': ['CO2', 'H'], 'index': [0, 3]}, 'https://meta.icos-cp.eu/objects/StAoU7ciukup4HmijO4J92sV': {'cols': ['H', 'H2O'], 'index': [1, 4]}, 'https://meta.icos-cp.eu/objects/9ISyDWIvPhNwbFGaqp6rwSP3': {'cols': ['H2O', 'CO2'], 'index': [2, 5]}}


In [13]:
def group_ts(var_tuple_ls: list = None, **kwargs) -> IcosFrame:
    """
    Returns a pandas dataframe df with an ICOS timeseries
    and attached metadata (an instance of IcosFrame which
    inherits pandas dataframe). In case there are more than
    one matching products, the latest will be returned.

    Parameters
    ----------
    var_tuple_ls a list of tuples
        each tuple in the list is assumed to be of the form
            (<variable>, <pid>)
        where the <variable> belongs to the ICOS product/file with pid <pid>

    Returns
    -------
    IcosFrame (see above)
    """

    # 1. Before we merge the data we need to take care of the order of the variables
    order_dict = {}
    for i in range(len(var_tuple_ls)):
        v, p = var_tuple_ls[i]
        if p in order_dict.keys():
            order_dict[p]['dobj'].append(Dobj(p))
            order_dict[p]['cols'].append(v)
            order_dict[p]['index'].append(i)
        else:
            order_dict[p]  = {'cols': [v], 'index': [i], 'dobj': [Dobj(p)]}
    
    # 2. We might also have to rename duplicate variables.
    products = set()
    stations = set()
    if len(order_dict.keys()) > 1: 
        # in this case we will rename columns
        for p in order_dict.keys():            
            p_meta = IcosFrame.trim_icos_meta(order_dict[p]['dobj'])
            products.add(p_meta['product'])
            stations.add(p_meta['stationId'])
            order_dict[p]['product'] = p_meta['product']
            order_dict[p]['stn_id'] = p_meta['stationId']
    if len(products) > 1:
        for p in order_dict.keys():
            prod = order_dict[p]['product']
            order_dict[p]['columns'] = [f'{prod}_{c}' for c in order_dict[p]['cols']]
    if len(stations) > 1:
        for p in order_dict.keys():
            stn_id = order_dict[p]['stn_id']
            if 'columns' in order_dict[p].keys():
                order_dict[p]['columns'] = [f'{stn_id}_{c}' for c in order_dict[p]['columns']]
            else:
                order_dict[p]['columns'] = [f'{stn_id}_{c}' for c in order_dict[p]['cols']]
    
    # 3. Gather data.
    for p in order_dict.keys():
        do = order_dict[p]['dobj']
        df = do.data
        
            
        
            
            
        
    print(order_dict)         
    
    
#     stations = {x[2] for x in var_tuple_ls}
#     st_df = self.stations_df

#     data_dict = {}
#     for st_id in stations:
#         data_dict[st_id] = {}
#         data_dict[st_id]['st_name'] = st_df.loc[(st_df.id ==
#                                                  st_id)].name.values[0]
#         files = {x[1] for x in var_tuple_ls if x[2] == st_id}
#         for spec in files:
#             data_dict[st_id][spec] = {}
#             pid = st_df.loc[(st_df.id == st_id) & (
#                     st_df.specLabel == spec)].dobj.values[0]

#             # variables of pid
#             var_ls = [x[0] for x in var_tuple_ls if
#                       x[2] == st_id and x[1] == spec]
#             data_dict[st_id][spec]['vars'] = var_ls

#             df = self.get_ts(pid=pid, col_ls=var_ls)
#             #do = Dobj(pid)
#             #df = do.data
#             #cols = var_ls
#             #if 'TIMESTAMP' not in var_ls:
#             #    cols.append('TIMESTAMP')

#             data_dict[st_id][spec]['data'] = df   #df[cols]

#     return data_dict

"""               col_meta_ls = do.meta['specificInfo'][
                    'columns'] if 'specificInfo' in do.meta.keys() and 'columns' in \
                                  do.meta['specificInfo'] else None

                var_meta = {}
                for v in var_ls:
                    for col in col_meta_ls:
                        if col['label'] == v:
                            var_meta[v] = {}
                            try:
                                var_meta[v]['unit'] = col['valueType']['unit']
                            except:
                                var_meta[v]['unit'] = 'No unit'
                            try:
                                label = col['valueType']['quantityKind'][
                                    'label']
                                var_meta[v]['label'] = label[
                                                           0].upper() + label[
                                                                        1:]
                            except:
                                var_meta[v]['label'] = ''
                data_dict[stn][spec]['meta'] = var_meta

        # Rename columns if there are stations or products involved
        # Are the more than 1 station?
        if len(data_dict.keys()) > 1:
            for stn in data_dict.keys():
                stn_short = data_dict[stn]['id']

                # are there more than one speces?
                if len(data_dict[stn].keys()) > 2:
                    for spec in data_dict[stn]:
                        if spec != 'id':
                            spec_short = spec.split(' ')[-1]

                            new_col_names = {
                                var: f'{stn_short}_{spec_short}_{var}'
                                for var in data_dict[stn][spec]['vars']
                                if var != 'TIMESTAMP'}
                            # new_col_names['TIMESTAMP'] = 'TIMESTAMP'

                            df = data_dict[stn][spec]['data']
                            data_dict[stn][spec]['data'] = df.rename(
                                columns=new_col_names)

                else:
                    for spec in data_dict[stn]:
                        if spec != 'id':
                            new_col_names = {var: f'{stn_short}_{var}'
                                             for var in
                                             data_dict[stn][spec]['vars']
                                             if var != 'TIMESTAMP'}
                            # "new_col_names['TIMESTAMP'] = 'TIMESTAMP'

                            df = data_dict[stn][spec]['data']
                            data_dict[stn][spec]['data'] = df.rename(
                                columns=new_col_names)
        else:
            for stn in data_dict.keys():
                # are there more than one speces?
                if len(data_dict[stn].keys()) > 2:
                    for spec in data_dict[stn]:
                        if spec != 'id':
                            spec_short = spec.split(' ')[-1]
                            new_col_names = {var: f'{spec_short}_{var}'
                                             for var in
                                             data_dict[stn][spec]['vars']
                                             if var != 'TIMESTAMP'}

                            df = data_dict[stn][spec]['data']
                            data_dict[stn][spec]['data'] = df.rename(
                                columns=new_col_names)

        return data_dict
"""

'               col_meta_ls = do.meta[\'specificInfo\'][\n                    \'columns\'] if \'specificInfo\' in do.meta.keys() and \'columns\' in                                   do.meta[\'specificInfo\'] else None\n\n                var_meta = {}\n                for v in var_ls:\n                    for col in col_meta_ls:\n                        if col[\'label\'] == v:\n                            var_meta[v] = {}\n                            try:\n                                var_meta[v][\'unit\'] = col[\'valueType\'][\'unit\']\n                            except:\n                                var_meta[v][\'unit\'] = \'No unit\'\n                            try:\n                                label = col[\'valueType\'][\'quantityKind\'][\n                                    \'label\']\n                                var_meta[v][\'label\'] = label[\n                                                           0].upper() + label[\n                                          

In [32]:
a = set()
a.add(8) #dir(a)
a.add(8)
a

{8}

In [33]:
len(None)

TypeError: object of type 'NoneType' has no len()